# Module 4: Feature Engineering & Selection

**Course**: End-to-End Machine Learning (Datacamp)  
**Case Study**: CardioCare Heart Disease Prediction  
**Author**: Seif

---

## Overview

Feature engineering and selection are crucial steps that build upon data preparation.

In this module, we cover:
1. What is feature engineering?
2. Normalization (scaling to 0-1)
3. Standardization (mean=0, variance=1)
4. What constitutes a good feature?
5. Feature selection with sklearn
6. Random Forest-based feature importance

**Remember**: More features isn't always better — it's about selecting the RIGHT features!

---

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

%matplotlib inline

## 1. What is Feature Engineering?

**Feature engineering** is the process of creating features that enhance ML model performance.

### Benefits:
- ✅ **Simpler models** — easier deployment and maintenance
- ✅ **Faster training** — fewer or more relevant features speed up learning
- ✅ **Better interpretability** — easier to explain model decisions
- ✅ **Improved performance** — capture essential aspects of data

### Two Main Approaches:
1. **Modify existing features** — transform, scale, or combine current features
2. **Create new features** — engineer completely new variables from raw data

**Key Principle**: More features ≠ better. Select features that capture essential aspects of the data.

## 2. Normalization

**Normalization** scales numeric features to a range of **0 to 1**.

### Why Normalize?
- Ensures no feature dominates the model due to its scale
- Essential when features have different ranges
- Required for scale-sensitive algorithms

### When to Use:
- **K-Nearest Neighbors (KNN)** — distance-based, very sensitive to scale
- **Neural Networks** — gradient descent converges faster with normalized inputs
- **Algorithms with distance metrics** — clustering, SVM with RBF kernel

### Formula:
$$x_{normalized} = \frac{x - x_{min}}{x_{max} - x_{min}}$$

In [ ]:
# Normalization using sklearn
from sklearn.preprocessing import Normalizer

# Create a normalizer object
normalizer = Normalizer()

# Normalize the DataFrame (assuming heart_disease_df is loaded)
# normalized_df = normalizer.fit_transform(heart_disease_df)

# Returns a normalized version where all features are scaled to 0-1 range

## 3. Standardization

**Standardization** scales features to have a **mean of 0** and **variance of 1**.

### Why Standardize?
- Centers data around zero
- Makes variance comparable across features
- Helps algorithms that assume normally distributed inputs

### When to Use:
- **Support Vector Machines (SVM)** — assumes features are centered
- **Linear Regression** — improves numerical stability
- **Principal Component Analysis (PCA)** — requires standardized data
- **Algorithms sensitive to variance** — logistic regression, ridge/lasso

### Formula:
$$x_{standardized} = \frac{x - \mu}{\sigma}$$

Where $\mu$ is the mean and $\sigma$ is the standard deviation.

In [ ]:
# Standardization using sklearn
from sklearn.preprocessing import StandardScaler

# Create a standard scaler object
scaler = StandardScaler()

# Standardize the DataFrame
# standardized_df = scaler.fit_transform(heart_disease_df)

# Returns a standardized version where features have mean=0 and variance=1

---

## Hands-On Exercise: Proper Standardization (Avoiding Data Leakage)

⚠️ **Critical**: Always fit the scaler on training data only, then transform both train and test sets.

In [ ]:
# Standardize 'age' on the training set and use the same standardizer to transform the 'age' column of the test set to avoid data leakage
standardizer = StandardScaler()
X_train['age'] = standardizer.fit_transform(X_train['age'].values.reshape(-1,1))
X_test['age'] = standardizer.transform(X_test['age'].values.reshape(-1,1))

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(X_train['age'], bins=30, alpha=0.5, label='Standardized')
plt.legend(prop={'size': 16})
plt.title('Histogram with Standardized Age')
plt.xlabel('Standardized Age')
plt.ylabel('Count')
plt.show()

### Why This Code Prevents Data Leakage:

1. **`fit_transform()` on training data only**  
   - Calculates mean and std from X_train
   - Applies standardization to X_train
   
2. **`transform()` on test data**  
   - Uses the SAME mean and std from training data
   - Does NOT recalculate statistics from X_test
   
3. **`.values.reshape(-1,1)`**  
   - Converts pandas Series to numpy array
   - Reshapes to 2D array (required by StandardScaler)
   - `-1` means "infer dimension", `1` means single column

### What the histogram shows:
After standardization, age values are centered around 0 with most values between -2 and 2 (within 2 standard deviations).

---

### 📊 Normalization vs. Standardization

| Aspect | Normalization | Standardization |
|--------|---------------|-----------------|
| **Range** | 0 to 1 | Unbounded (centered at 0) |
| **Formula** | $(x - min) / (max - min)$ | $(x - \mu) / \sigma$ |
| **Best for** | KNN, Neural Nets | SVM, Linear models |
| **Sensitivity** | Affected by outliers | Less affected by outliers |
| **Use case** | Bounded range needed | Normal distribution assumed |

## 4. What Constitutes a Good Feature?

Good features improve prediction accuracy. To select good features, we need to consider:

### ✅ Relevance
Features should be **relevant** to the prediction task.

**Good**: Age, cholesterol, blood pressure → directly related to heart disease  
**Bad**: Weather on appointment day → no bearing on diagnosis

### ✅ Non-redundancy
Avoid multiple features representing **similar metrics**.

**Redundant**: Both "age in years" AND "age in months" — capture the same information  
**Better**: Choose one (typically age in years)

### ✅ Orthogonality
Good features should be **dissimilar** (perpendicular/orthogonal) to each other.

- Features that are highly correlated don't add new information
- Orthogonal features capture different aspects of the data
- Example: Age and cholesterol are orthogonal (independent measurements)

## 5. Feature Selection with sklearn

**sklearn.feature_selection** provides a robust toolbox for selecting significant, non-redundant features.

### ⚠️ Important: Split Data First!
Always split your data **before** feature selection to avoid **data leakage**.

```python
# Split first to prevent test data from influencing feature selection
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

Data leakage occurs when the model is exposed to test data during training or feature selection, leading to overly optimistic performance estimates.

## 6. SelectFromModel with Random Forest

**SelectFromModel** selects features based on importance weights from a fitted model.

### How it Works:
1. Train a Random Forest classifier
2. Random Forest calculates feature importance
3. Eliminate features considered irrelevant
4. Keep only features that improve predictions

In [ ]:
# Step 1: Split the data first
# X_train, X_test, y_train, y_test = train_test_split(
#     heart_disease_x, heart_disease_y, test_size=0.2, random_state=42
# )

In [ ]:
# Step 2: Create and train a Random Forest classifier
# model = RandomForestClassifier(
#     n_jobs=-1,           # Use all available processors
#     class_weight='balanced',  # Balance class frequencies
#     max_depth=5,         # Limit tree depth to 5
#     random_state=42
# )

# # Fit the model on training data
# model.fit(X_train, y_train)

In [ ]:
# Step 3: Use SelectFromModel to select important features
# from sklearn.feature_selection import SelectFromModel

# selector = SelectFromModel(model, prefit=True)
# # prefit=True means the model has already been fitted

# # Get the boolean mask of selected features
# selected_features = selector.get_support()

# print("Selected features:", selected_features)
# # Returns: [True, False, True, True, False, ...]
# # True = feature is important, False = feature is not important

In [ ]:
# Step 4: Transform the data to keep only selected features
# X_train_selected = selector.transform(X_train)
# X_test_selected = selector.transform(X_test)

# # Or get the selected feature names
# feature_names = heart_disease_x.columns
# selected_feature_names = feature_names[selected_features]
# print("Important features:", selected_feature_names.tolist())

---

## Hands-On Exercise: Random Forest Feature Selection

Complete workflow for selecting important features using Random Forest.

In [ ]:
from sklearn.feature_selection import SelectFromModel

# Define the random forest model and fit to the training data
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
rf.fit(X_train, y_train)

In [ ]:
# Define the feature selection object
model = SelectFromModel(rf, prefit=True)

# Transform the training features
X_train_transformed = model.transform(X_train)

In [ ]:
original_features = heart_disease_df.columns[:-1]
print(f"Original features: {original_features}")

# Select the features deemed important by the SelectFromModel
features_bool = model.get_support()

selected_features = original_features[features_bool]
print(f"\nSelected features: {selected_features}")

In [ ]:
feature_importance = pd.DataFrame({
    "feature": selected_features,
    "importance": rf.feature_importances_[features_bool]
})
plt.figure(figsize=(10, 6))
plt.barh(feature_importance["feature"], feature_importance["importance"])
plt.show()

### Understanding This Exercise:

1. **Train Random Forest**  
   - Fits on training data to learn feature importance
   - Uses balanced class weights to handle imbalance
   
2. **Create SelectFromModel**  
   - `prefit=True` indicates the model is already fitted
   - Automatically selects features above importance threshold
   
3. **Transform Data**  
   - `model.transform(X_train)` removes low-importance features
   - Returns only the columns deemed important
   
4. **Get Feature Names**  
   - `model.get_support()` returns boolean array [True, False, True, ...]
   - `original_features[features_bool]` filters to selected features only
   
5. **Visualize Importance**  
   - Horizontal bar chart shows which features matter most
   - Longer bars = higher importance for predicting heart disease

### Expected Output:
```
Original features: Index(['age', 'sex', 'cp', 'trestbps', 'chol', ...])
Selected features: Index(['age', 'cp', 'thalach', 'oldpeak', ...])
```

The bar chart reveals which patient health indicators are most predictive of heart disease.

---

### Understanding Random Forest Parameters

**`n_jobs=-1`**  
Use all available CPU processors for parallel computation (speeds up training)

**`class_weight='balanced'`**  
Automatically adjusts weights inversely proportional to class frequencies  
Helps with imbalanced datasets (e.g., more patients without disease than with)

**`max_depth=5`**  
Limits the maximum depth of each tree to 5 levels  
Prevents overfitting by restricting tree complexity

**`prefit=True`**  
Tells `SelectFromModel` that the model has already been fitted  
No need to fit again inside the selector

---

## Key Takeaways

1. **Feature Engineering Enhances Performance**: Proper feature engineering simplifies models and improves accuracy
2. **Normalization vs. Standardization**: Choose based on algorithm requirements
   - Normalization (0-1) → KNN, Neural Networks
   - Standardization (mean=0) → SVM, Linear models
3. **Quality > Quantity**: More features ≠ better. Focus on relevant, non-redundant features
4. **Avoid Data Leakage**: Always split data before feature selection
5. **Random Forest for Selection**: Automatically identifies important features based on predictive power
6. **Orthogonality Matters**: Features should capture different aspects of the data

---

## Feature Engineering Workflow

1. **Clean the data** (Module 3)
2. **Split train/test** (avoid data leakage)
3. **Apply scaling** (normalization or standardization)
4. **Select features** (Random Forest, correlation analysis, domain knowledge)
5. **Validate selection** (check performance on test set)
6. **Iterate** (refine based on results)

---

## References
- Datacamp: End-to-End Machine Learning Course
- Video 4: Feature Engineering and Selection
- [Scikit-learn Preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html)
- [Scikit-learn Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)